## Create The ONNX File

In [1]:
from special_neurons import get_most_negative_sets
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

model_name = 'EleutherAI/pythia-160m'
model_name = 'EleutherAI/pythia-70m'
# model_name = 'EleutherAI/gpt-neo-1.3B'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model_og = AutoModelForCausalLM.from_pretrained(model_name).to(device)
most_neg = get_most_negative_sets(model_og)
most_neg[0].prev_layer_name, most_neg[0].linear_layer_name

('gpt_neox.layers.0.mlp', 'gpt_neox.layers.0.mlp.dense_h_to_4h')

In [2]:
# Our max_norm is none
print(model_og.gpt_neox.embed_in.max_norm, model_og.gpt_neox.embed_in.norm_type, model_og.gpt_neox.embed_in.scale_grad_by_freq)

None 2.0 False


In [3]:
model_og.gpt_neox.layers[0].attention?

Signature:       model_og.gpt_neox.layers[0].attention(*args, **kwargs)
Type:            GPTNeoXAttention
String form:    
GPTNeoXAttention(
           (rotary_emb): GPTNeoXRotaryEmbedding()
           (query_key_value): Linear(in_feature <...> features=512, out_features=512, bias=True)
           (attention_dropout): Dropout(p=0.0, inplace=False)
           )
File:            ~/.local/lib/python3.9/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py
Docstring:       <no docstring>
Class docstring:
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def fo

In [4]:
model_og.gpt_neox.layers[0].input_layernorm

LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [5]:
embd_matrix = model_og.gpt_neox.embed_in.weight
embd_matrix.shape

torch.Size([50304, 512])

In [6]:
def get_input_after_tokenizer(inp: str):
		tokenizer = AutoTokenizer.from_pretrained(model_name)
		inp_out = tokenizer(inp, return_tensors="pt")
		print(inp_out)
		inputs = inp_out['input_ids'].squeeze(0)
		# print(inputs, inputs.shape)
		# TODO: IDK ABOUT WHATS GOING ON W/ TOKEN SIZE VS Vocab Size
		# Vocab size is 50204 and inp size is 50304
		one_hot = torch.zeros((inputs.shape[0], 50304), dtype=torch.int)
		for i in range(inputs.shape[0]):
			one_hot[i, inputs[i]] = 1
		# one_hot[inputs['input_ids'][0, 0]] = 1
		return one_hot#, inp_out['attention_mask']
get_input_after_tokenizer('hello world')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [7]:
inps_one_hot = get_input_after_tokenizer("hello world")
inps_one_hot.shape, inps_one_hot.nonzero()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


(torch.Size([2, 50304]),
 tensor([[    0, 25521],
         [    1,  1533]]))

In [52]:
import transformers.models.gpt_neox as gpt_neox
N_TOKENS = inps_one_hot.shape[0]
attention_mask = torch.ones((1, N_TOKENS), dtype=torch.int)


class FixedAttentionMask(torch.nn.Module):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.attn = model_og.gpt_neox.layers[0].attention
    
    def _attn(self, query, key, value, attention_mask=None, head_mask=None):
        # q, k, v: [bs, num_attention_heads, seq_len, attn_head_size]
        # compute causal mask from causal mask buffer
        num_attention_heads, query_length, attn_head_size = query.size()
        key_length = key.size(-2)

        # dynamically increase the causal mask with the key length, if needed.
        if key_length > self.attn.bias.shape[-1]:
            self.attn._init_bias(key_length, device=key.device)
        causal_mask = self.attn.bias[:, :, key_length - query_length : key_length, :key_length]

        query = query.view(num_attention_heads, query_length, attn_head_size)
        key = key.view(num_attention_heads, key_length, attn_head_size)
        attn_scores = torch.zeros(
            num_attention_heads,
            query_length,
            key_length,
            dtype=query.dtype,
            device=key.device,
        )
        attn_scores = torch.baddbmm(
            attn_scores,
            query,
            key.transpose(1, 2),
            beta=1.0,
            alpha=self.attn.norm_factor,
        )
        attn_scores = attn_scores.view(num_attention_heads, query_length, key_length)

        mask_value = torch.finfo(attn_scores.dtype).min
        # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
        # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
        mask_value = torch.tensor(mask_value, dtype=attn_scores.dtype).to(attn_scores.device)
        attn_scores = torch.where(causal_mask, attn_scores, mask_value)

        if attention_mask is not None:
            # Apply the attention mask
            attn_scores = attn_scores + attention_mask

        attn_weights = torch.nn.functional.softmax(attn_scores, dim=-1)
        attn_weights = attn_weights.to(value.dtype)

        # Mask heads if we want to
        if head_mask is not None:
            attn_weights = attn_weights * head_mask

        attn_weights = self.attn.attention_dropout(attn_weights)

        attn_output = torch.matmul(attn_weights, value)
        return attn_output, attn_weights



    def forward(self, x):
        # TODO: IDK IF THIS IS RIGHT for POSITION IDS or ATTENTION MASK
        position_ids=torch.arange(N_TOKENS)
        # Compute QKV
        # Attention heads [seq_len, hidden_size] --> [seq_len, (np * 3 * head_size)]
        qkv = self.attn.query_key_value(x)[0]
        print(qkv.shape)

        # [seq_len, (num_heads * 3 * head_size)] --> [seq_len, num_heads, 3 * head_size]
        new_qkv_shape = qkv.size()[:-1] + (self.attn.num_attention_heads, 3 * self.attn.head_size)
        qkv = qkv.view(*new_qkv_shape)

        # [seq_len, num_attention_heads, 3 * head_size] --> 3 [num_attention_heads, seq_len, head_size]
        query = qkv[..., :self.attn.head_size].permute(1, 0, 2)
        key = qkv[..., self.attn.head_size:2 * self.attn.head_size].permute(1, 0, 2)
        value = qkv[..., 2 * self.attn.head_size:].permute(1, 0, 2)

        # Compute rotary embeddings on rotary_ndims
        query_rot = query[..., :self.attn.rotary_ndims]
        query_pass = query[..., self.attn.rotary_ndims:]
        key_rot = key[..., :self.attn.rotary_ndims]
        key_pass = key[..., self.attn.rotary_ndims:]

        # Compute token offset for rotary embeddings (when decoding)
        seq_len = key.shape[-2]
        cos, sin = self.attn.rotary_emb(value, seq_len=seq_len)


        def rotate_half(x):
            """Rotates half the hidden dims of the input."""
            x1 = x[..., : x.shape[-1] // 2]
            x2 = x[..., x.shape[-1] // 2 :]
            return torch.cat((-x2, x1), dim=-1)

        def apply_rotary_embed(q, k, cos, sin, position_ids):
            cos = cos[position_ids]
            sin = sin[position_ids]
            q_embed = (q * cos) + (rotate_half(q) * sin)
            k_embed = (k * cos) + (rotate_half(k) * sin)
            return q_embed, k_embed
        print(query_rot.shape, key_rot.shape, cos.shape, sin.shape, position_ids.shape)
        query, key = apply_rotary_embed(query_rot, key_rot, cos, sin, position_ids)
        query = torch.cat((query, query_pass), dim=-1)
        key = torch.cat((key, key_pass), dim=-1)

        # Cache QKV values
        # if has_layer_past:
        #     past_key = layer_past[0]
        #     past_value = layer_past[1]
        #     key = torch.cat((past_key, key), dim=-2)
        #     value = torch.cat((past_value, value), dim=-2)
        present = None

        # Compute attention
        attn_output, attn_weights = self._attn(query, key, value, attention_mask, None)

        # Reshape outputs
        attn_output = self.attn._merge_heads(attn_output, self.attn.num_attention_heads, self.attn.head_size)
        attn_output = self.attn.dense(attn_output)

        outputs = (attn_output, present)
        # if output_attentions:
        #     outputs += (attn_weights,)

        return outputs

        return self.attn(x, attention_mask=attention_mask, position_ids=torch.arange(N_TOKENS).unsqueeze(0))


embed_linear = torch.nn.Linear(
    embd_matrix.shape[0], embd_matrix.shape[1], bias=False)
embed_linear.weight = torch.nn.Parameter(embd_matrix.T)


class SimplfiedLayerNorm(torch.nn.Module):
    def __init__(self, layernorm: torch.nn.LayerNorm) -> None:
        super().__init__()
        self.weight = layernorm.weight
        self.bias = layernorm.bias
        self.eps = torch.nn.Linear(512, 512)
        self.eps.weight = torch.nn.Parameter(torch.eye(512))
        self.eps.bias = torch.nn.Parameter(torch.ones(512) * 1e-5)
        # TODO: not constant
        self.ones_linear = torch.nn.Linear(512, 512, bias=False)
        self.ones_linear.weight = torch.nn.Parameter(torch.ones((512, 512)))
        self.ones_linear_neg = torch.nn.Linear(512, 512, bias=False)
        self.ones_linear_neg.weight = torch.nn.Parameter(-1 * torch.ones((512, 512)))

    def forward(self, x):
        # TODO: I think that this can be made more efficient
        expectation_neg = self.ones_linear_neg(x)
        variance = self.ones_linear((((x + expectation_neg) * (x + expectation_neg))))
        radical = self.eps(variance)
        denom = torch.sqrt(radical)
        x = x + expectation_neg
        # TODO: DENOM HAS PROVLEMS
        return x
        x = x / denom
        return x
        x = x * self.weight
        x = x + self.bias

        return x


class ModelSel(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.embed_linear = embed_linear
        self.layer_norm = SimplfiedLayerNorm(model_og.gpt_neox.layers[0].input_layernorm)
        self.attn = FixedAttentionMask()

    def forward(self, x):
        x = self.embed_linear(x)
        x = self.layer_norm(x)
        # HRMM Unsqueeze no good
        x = x.view((1, *x.shape))
        x = self.attn(x)
        return x[0]

# G
# # TODO: add residuals?
# model_sel = torch.nn.Sequential(
#     # model_og.gpt_neox.embed_in,
#     embed_linear,
#     # model_og.gpt_neox.emb_dropout, # we have p = 0.0 and thus useless
#     model_og.gpt_neox.layers[0].input_layernorm,
#     FixedAttentionMask(),
#     # TODO: VERIFY THIS JAZZ
# )


model_sel = ModelSel()
model_sel

ModelSel(
  (embed_linear): Linear(in_features=50304, out_features=512, bias=False)
  (layer_norm): SimplfiedLayerNorm(
    (eps): Linear(in_features=512, out_features=512, bias=True)
    (ones_linear): Linear(in_features=512, out_features=512, bias=False)
    (ones_linear_neg): Linear(in_features=512, out_features=512, bias=False)
  )
  (attn): FixedAttentionMask(
    (attn): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
      (dense): Linear(in_features=512, out_features=512, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
    )
  )
)

In [50]:
inps_one_hot.shape, embed_linear.weight.shape
inps_one_hot_formatted = inps_one_hot.float()
# embed_linear.forward(inps_one_hot).shape
model_sel(inps_one_hot_formatted)[0].shape

torch.Size([2, 1536])
torch.Size([8, 2, 16]) torch.Size([8, 2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])


AttributeError: 'FixedAttentionMask' object has no attribute 'dense'

In [53]:
torch.onnx.export(model_sel, inps_one_hot_formatted,
                  'model_sel.onnx', verbose=False, opset_version=12)

torch.Size([2, 1536])
torch.Size([8, 2, 16]) torch.Size([8, 2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/tmp/ipykernel_35776/4286207849.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if key_length > self.attn.bias.shape[-1]:
/tmp/ipykernel_35776/4286207849.py:44: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask_value = torch.tensor(mask_value, dtype=attn_scores.dtype).to(attn_scores.device)


AttributeError: 'FixedAttentionMask' object has no attribute 'dense'

In [ ]:
import onnx

# Load the ONNX model
model = onnx.load("model_sel.onnx")

# Get the names of input and output nodes
input_names = [input.name for input in model.graph.input]
output_names = [output.name for output in model.graph.output]

print("Input Names:", input_names)
print("Output Names:", output_names)

Input Names: ['onnx::MatMul_0']
Output Names: ['16']


## Use Mariboupy

In [ ]:
# !export PYTHONPATH="$PYTHONPATH:/home/lev/code/research/ai/dictator/Marabou"
# !export PYTHONPATH="$PYTHONPATH:/home/lev/code/research/ai/dictator/Marabou/maraboupy"
# !pip install onnx onnxruntime maraboupy --upgrade

In [ ]:
!pip install onnx onnxruntime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import sys
import numpy as np

## %
# Path to Marabou folder if you did not export it

# TODO: this is a hack, fix it
# TODO: add Marabou installation details to readme
# sys.path.append('/home/lev/code/research/softmax_bound/bounding-softmax/Marabou')
sys.path.append('/home/lev/code/research/ai/dictator/Marabou')
from maraboupy import Marabou

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
options = Marabou.createOptions(verbosity = 1)
print("Simple Attention Head")
filename = "model_sel.onnx"
network = Marabou.read_onnx(filename)#, inputNames=inputNames, outputNames=[outputName])

Simple Attention Head


## Play around with basic constraints